ライブラリのインポート

In [16]:
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta

定数宣言

In [63]:


HOWTIME = 36      #開催回数
HOWDAYS = 12      #１開催につき何日目
RACENUM = 12      #１日何レースか

#開催場コード
SAPORO = "01"     #札幌
HAKODATE = "02"   #函館
HUKUSHIMA = "03"  #福島
NIGATA = "04"     #新潟
TOKYO = "05"      #東京
NAKAYAMA = "06"   #中山
TYUKYO = "07"     #中京
KYOTO = "08"      #京都
HANSHIN = "09"    #阪神
OGURA = "10"      #小倉

#上記のコードをまとめたリスト
race_venue_list = [
  SAPORO,HAKODATE,HUKUSHIMA,NIGATA,TOKYO,NAKAYAMA,TYUKYO,KYOTO,HANSHIN,OGURA
  ]

In [89]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
# import scrayping
import time
from datetime import datetime, timedelta


def scrayping():
  #変数宣言
  year = "2023"        #任意の年数
  race_id = ""         #宣言
  error = "true"       #エラーフラグ
  result_list = []     #result_rankingの子要素
  race_list = []       #race_infoの子要素
  pay_list = []        #result_payの子要素
  result_ranking = {}  #total_infoの子要素
  race_info = {}       #total_infoの子要素
  resulut_pay = {}     #total_infoの子要素
  total_info = []      #すべてまとめたリスト

  #指定した年から現在の日付までの全レースを取得
  for where in race_venue_list:
    error = "true"
    for howtime in range(HOWTIME):
      for howdays in range(HOWDAYS):
        for race_num in range(RACENUM):

          #文字に変換
          ht = str(howtime+1)
          hd = str(howdays+1)
          rn = str(race_num+1)

          #1桁の数字を0付きの2桁の数字に変換
          race_id = year + where + ('0' + ht if len(ht) != 2 else ht) + ('0' + hd if len(hd) != 2 else hd) + ('0' + rn if len(rn) != 2 else rn)

          #URLを作成
          url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}&rf=race_list"

          #スクレイピング
          res = requests.get(url)
          res.encoding = "EUC-JP"
          soup = BeautifulSoup(res.text, "html.parser")
          
          list = soup.find_all("table")
          title = soup.find_all("title")

          #正常にスクレイピングが行われた場合
          if list != []:

            #必要な情報を取り出す
            list = list[0].text
            list = list.split()
            
            racename = soup.find("div",class_="RaceName").text.split() #レース名
            rd01 = soup.find("div",class_="RaceData01").text           #発走時刻,馬場(種類),距離,[右回り、左回り],天気,馬場状態
            rd01 = re.split('[/\n]',rd01)                              #['', '09:50発走 ', ' 芝1200m (右\xa0A)', '', ' 天候:晴', '', ' 馬場:良', '']
            rd02 = soup.find("div",class_="RaceData02").text.split()   #第何回,開催場,何日目,[],[],[],[],何頭,[]
            day = soup.find("dd",class_="Active").text                 #x月x日(x曜日)
            
            #race_list順番(データベース定義書の順番)
            order = [day,racename[0],rd01[1],rd01[2][2:7],rd02[7],rd01[2][1:2],rd02[1],rd01[4][4:5],rd01[2][8:],rd01[6][4:5]]

            #race_listに格納
            for x in order:
              race_list.append(x)

            # error = "false"
            #1秒待つ
            time.sleep(0.1)

            # #上位５馬とレースタイトル表示
            # z = 14
            # for x in range (5):
            #   for y in range (2):
            #     if x == 0:
            #       # print(f"{list[3 + z + y]}",end=' ')
            #       result_list.append(list[3 + z + y])
            #     else:
            #       # print(f"{list[2 + z + y]}",end=' ')
            #       result_list.append(list[2 + z + y])
            #   z+=15

            #改行
            # print()
            # print()

            #各辞書型の配列にリストを格納する
            result_ranking[race_id] = result_list
            race_info[race_id] = race_list
            resulut_pay[race_id] = pay_list

            # print(result_ranking)
            # print(result_list)

          else:
            error = "false"
            break
          #親要素に追加したら初期化
          result_list = []
          race_list = []
          pay_list = []
          print(race_info)
        if error == "false" :
          break
        return result_ranking
      if error == "false" :
        break
  
  #スクレイピングしたデータをすべてtotal_infoに格納
  total_info.append(result_ranking)
  total_info.append(race_info)
  total_info.append(resulut_pay)

  #呼び出し元に渡す
  return total_info

#関数呼び出し
scrayping()



{'202301010101': ['7月22日(土)', '2歳未勝利', '09:50発走 ', '1200m', '8頭', '芝', '札幌', '晴', '(右\xa0A)', '良']}
{'202301010101': ['7月22日(土)', '2歳未勝利', '09:50発走 ', '1200m', '8頭', '芝', '札幌', '晴', '(右\xa0A)', '良'], '202301010102': ['7月22日(土)', '3歳未勝利', '10:25発走 ', '1000m', '12頭', 'ダ', '札幌', '晴', '(右)', '良']}
{'202301010101': ['7月22日(土)', '2歳未勝利', '09:50発走 ', '1200m', '8頭', '芝', '札幌', '晴', '(右\xa0A)', '良'], '202301010102': ['7月22日(土)', '3歳未勝利', '10:25発走 ', '1000m', '12頭', 'ダ', '札幌', '晴', '(右)', '良'], '202301010103': ['7月22日(土)', '3歳未勝利', '10:55発走 ', '1700m', '14頭', 'ダ', '札幌', '晴', '(右)', '良']}
{'202301010101': ['7月22日(土)', '2歳未勝利', '09:50発走 ', '1200m', '8頭', '芝', '札幌', '晴', '(右\xa0A)', '良'], '202301010102': ['7月22日(土)', '3歳未勝利', '10:25発走 ', '1000m', '12頭', 'ダ', '札幌', '晴', '(右)', '良'], '202301010103': ['7月22日(土)', '3歳未勝利', '10:55発走 ', '1700m', '14頭', 'ダ', '札幌', '晴', '(右)', '良'], '202301010104': ['7月22日(土)', '3歳未勝利', '11:25発走 ', '1500m', '13頭', '芝', '札幌', '晴', '(右\xa0A)', '良']}
{'202301010101': ['7月22日(

KeyboardInterrupt: 